DACON CV

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
from glob import glob
import numpy as np
from PIL import Image

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [28]:
%cd /content/gdrive/My Drive/dacon/img/

/content/gdrive/My Drive/dacon/img


In [18]:
tr_imgs = []
tr_labels = []

In [ ]:
for fname in glob('*') :
  for file in glob(fname + '/*.jpg') :
    img = Image.open(file)
    img_array = np.array(img)
    tr_imgs.append(img_array)
    label = fname.split('/')[0]
    tr_labels.append(label)

In [21]:
%cd /content/gdrive/My Drive/dacon/img/test

/content/gdrive/My Drive/dacon/img/test


In [22]:
te_imgs = []
fnames = []

In [23]:
for file in glob('*') :
  fnames.append(file)
  #img = Image.open(file)
  #img_array = np.array(img)
  #te_imgs.append(img_array)

In [ ]:
len(te_imgs)

10089

In [34]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
!pip install -q -U tensorflow-addons
import tensorflow_addons as tfa

In [6]:
tr_imgs = np.load('tr.npy')
te_imgs = np.load('te.npy')

In [7]:
tr_labels = np.load('tr_label.npy')

In [8]:
X = tr_imgs / 255.0
target = te_imgs / 255.0

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
le = LabelEncoder()

In [11]:
y = le.fit_transform(tr_labels)

In [12]:
X.shape

(50000, 32, 32, 3)

In [13]:
skf = StratifiedKFold(n_splits = 20, random_state = 42, shuffle = True)

In [35]:
reLR = ReduceLROnPlateau(patience = 3, verbose = 1, factor = 0.95)
es = EarlyStopping(patience = 5, verbose = 1, monitor = 'val_acc', mode = 'max')

sub_pred = np.zeros((target.shape[0], 10))

for i, (tr_idx, val_idx) in enumerate(skf.split(X, y)) :
  print('=' * 40)
  print(f'{i + 1}번째 학습 FOLD 학습 시작')
  tr_x, tr_y = X[tr_idx], y[tr_idx]
  val_x, val_y = X[val_idx], y[val_idx]

  mc = ModelCheckpoint(f'cv{i + 1}.h5', save_best_only = True, verbose = 1, monitor = 'val_acc', mode = 'max', save_weights_only = True)

  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape = (32, 32, 3)))
  model.add(BatchNormalization())
  model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPool2D((2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPool2D((2, 2)))
  model.add(Dropout(0.3))
  model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPool2D((2, 2)))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(128, activation = 'relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))
  model.add(Dense(64, activation = 'relu'))
  model.add(Dense(32, activation = 'relu'))
  model.add(Dense(10, activation = 'softmax'))

  model.compile(tfa.optimizers.RectifiedAdam(0.004), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

  history = model.fit(tr_x, tr_y, epochs = 1000, batch_size = 128, validation_data = (val_x, val_y), callbacks = [es, mc, reLR])

  model.load_weights(f'cv{i + 1}.h5')

  pred = model.predict(target) / skf.n_splits
  sub_pred += pred
  print(f'{i + 1}번째 학습 FOLD 학습 완료\n')

1번째 학습 FOLD 학습 시작
Epoch 1/1000
371/372 [============================>.] - ETA: 0s - loss: 1.6401 - acc: 0.3982
Epoch 1: val_acc improved from -inf to 0.16640, saving model to cv1.h5
372/372 [==============================] - 29s 47ms/step - loss: 1.6401 - acc: 0.3983 - val_loss: 3.2317 - val_acc: 0.1664 - lr: 0.0040
Epoch 2/1000
372/372 [==============================] - ETA: 0s - loss: 1.1730 - acc: 0.5786
Epoch 2: val_acc improved from 0.16640 to 0.51000, saving model to cv1.h5
372/372 [==============================] - 18s 47ms/step - loss: 1.1730 - acc: 0.5786 - val_loss: 1.5707 - val_acc: 0.5100 - lr: 0.0040
Epoch 3/1000
372/372 [==============================] - ETA: 0s - loss: 0.9916 - acc: 0.6469
Epoch 3: val_acc improved from 0.51000 to 0.64800, saving model to cv1.h5
372/372 [==============================] - 18s 48ms/step - loss: 0.9916 - acc: 0.6469 - val_loss: 1.0540 - val_acc: 0.6480 - lr: 0.0040
Epoch 4/1000
371/372 [============================>.] - ETA: 0s - loss: 0.88

KeyboardInterrupt: ignored

In [16]:
import pandas as pd

In [29]:
fnames = np.load('fnames.npy')

In [30]:
submission = pd.DataFrame({"id" : fnames, "target" : le.inverse_transform(np.argmax(sub_pred, axis = 1))}).sort_values(by = 'id')
submission.index = range(submission.shape[0])

In [31]:
submission.to_csv("20220301_1st.csv", index = False)